# Setup


### Setup: Import Libraries

In [16]:
# Modelling Libraries
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
# NLP Libraries
import nltk
nltk.download('punkt')
nltk.download("wordnet")
from nltk.stem import WordNetLemmatizer
# Utility Libraries
import pandas as pd
import random
import numpy as np
import json
import pickle
from pathlib import Path
import os
import utils


[nltk_data] Downloading package punkt to /Users/henrylao/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/henrylao/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### Setup: Project Paths

In [18]:
ROOT = utils.get_project_root()
INPUT = ROOT / "input" 
CACHE = ROOT / "model_cache" # directory for saving trained models
print("ROOT: ", os.listdir(ROOT))
print("INPUT: ", os.listdir(INPUT))
print("CACHE: ", os.listdir(CACHE))

ROOT:  ['model_cache', 'res', 'ref', 'input', 'models']
INPUT:  ['dzone-intents.json', 'oos-eval', 'tim-intents.json']
CACHE:  ['chatbot_model.h5', 'classes.pkl', 'words.pkl', 'oos-chatbot_model.h5', 'oos-words.pkl', 'oos-labels.pkl']


### Setup: Load Data 

In [ ]:
lemmatizer = WordNetLemmatizer()
# load data
with open( INPUT / "dzone-intents.json", "r") as fh:
    intents = json.load(fh)
print(intents.keys())
# the data is a list of dictionaries
print(type(intents['intents']))
print(type(intents['intents'][5]))
print(intents['intents'][0])

# Data Preprocessing
* Tokenization of the sentences
* Lemmatization of existing words to reduce all the words to their canonical form (same lexeme)
    Ex. played, plays, playing -> play

### Data Preprocessing: Tokenization

In [ ]:
ignore_chars = "! ? , .".split(" ")
# tokenization
def tokenize(intents:dict)->tuple:
    words, classes, documents = [], [], []
    for i in intents["intents"]:
        for p in i["patterns"]:
            word = nltk.word_tokenize(p)
            words.extend(word)
            documents.append((word, i['tag']))
            if i['tag'] not in classes:
                classes.append(i['tag'])
    return words, classes, documents

(words, classes, documents ) = tokenize(intents)

In [ ]:
documents[:3]

In [ ]:
words[:5]

In [ ]:
classes[:5]

Responses appear to never be used here. Atleast in forming the 

### Data Preprocessing: Lemmatization

In [ ]:
# lemmatization
# tranformation of words into a lemma form such that they can be reduced
# to canonical words
# Ex. play, played, plays, playing --> are all mean play
# Reason: reduce number of total words in the model's vocab
words = [lemmatizer.lemmatize(w.lower()) 
         for w in words if w not in ignore_chars]
words = sorted(list(set(words)))
classes = sorted(list(set(classes)))
# cache into binary files
with open(CACHE / "words.pkl", "wb") as fh:
    pickle.dump(words, fh)
with open(CACHE/"classes.pkl", "wb") as fh:
    pickle.dump(classes, fh)
print("Set Sizes")
print("Documents:",len(documents))
print("Classes:", len(classes))
print("Words:", len(words))


# Create Training and Testing Set
* What are we doing here with the documents and the words?
* How does the creation of the training dataset work here?
* Numeric conversion of input patterns into numbers
    1. Lemmatize words of the pattern
    2. Create the list of zeroes of the same value of the magnitude of the set of all words to be trained
    3. Set value to be 1 for indices in the list of zeroes in which the word exists in the pattern
    
    
* Numeric conversion of the input classes applying same method used in input pattern conversion

In [ ]:
# recall that an item in the documents consists of the word pattern + class
print("Entry:",documents[0]) # entry
print("Word pattern:",documents[0][0]) # word pattern
print("Class:",documents[0][1]) # class

In [ ]:
classes

In [ ]:
x = []
Y_empty = [0]*len(classes) 
# iterate over each entry in documents
for d in documents:
    print("Word pattern", d)
    bag = []
    word_patterns = d[0] # access the word pattern
    # lemmatize the entries in the selected word pattern
    lemma_word_patterns = [lemmatizer.lemmatize(w.lower()) for
                    w in word_patterns]
    print("Lemmatized:", word_patterns)
    if (len(lemma_word_patterns) == len(word_patterns)):
        print("Identified case in which the collection of words were reduced to their core meaning")
    # convert the lemmatized word pattern to a numerical value
    for _ in words:
        bag.append(1) if _ in lemma_word_patterns else bag.append(0)
        
    # create the label for the classes in numeric form
    Y_row = list(Y_empty) 
    # set the activation on the output row
    Y_row[classes.index(d[1])] = 1
    print("Numeric Class:",classes.index(d[1]))
    print("Y_row:",Y_row)
    print()
    x.append([bag, Y_row]) # add to the training set

random.shuffle(x) # not sure if this is necessary? why shuffle the data when we create the training data with all of it and there is no test set
x = np.array(x)
x_train, Y_train = list(x[:,0]), list(x[:,1])


In [ ]:
df_x_train  = pd.DataFrame(x_train)
print("Training input shape:",df_x_train.shape)
# x_train

In [ ]:
df_Y_train = pd.DataFrame(Y_train)
print("Training output shape",df_Y_train.shape)
# Y_train

Recall when the intents were loaded, that the size of documents was 47. 
The value of 47 represents the number of instances that are used to train the model. 
The value 87 is the number of features in the dataset ie the number of unique words that occur. 
9 is the number of classifcations for the entire dataset. 

# Create the Model
* https://keras.io/guides/sequential_model/
* https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense


### Sequential Model
* <b>When to use:</b> 
    - nice for a <b>plain stack of layers</b> ie each layer has <b>exactly one input tensor and one output tensor</b>
* <b>When NOT to use:</b>
    - model has multiple inputs and/or multiple outputs
    - any of layers has multiple inputs and/or multiple outputs
    - need layer sharing
    - want non-linear topology (ie a residual connection, a multi-branch model)
    
    
### Tensor
* Link to <a href="Tensor Basics with Numpy.ipynb">Tensor Basics Notebook</a>
<img src="res/n-order-tensor.jpeg" ><br> 


### Perceptron

### Layer (Dense)


### Rectified Linear Activation Function (ReLU) & Softmax
* https://machinelearningmastery.com/rectified-linear-activation-function-for-deep-learning-neural-networks/
* https://machinelearningmastery.com/softmax-activation-function-with-python/
* https://deepai.org/machine-learning-glossary-and-terms/softmax-layer
* Why softmax for the output activation? 
* Why ReLU for the input activation?

### Stochastic Gradient Descent (SGD)


In [ ]:
# use a deep neural network model
model = Sequential()
# add the 1st layer -- an input layer
model.add(Dense(128, input_shape=(len(x_train[0]),), activation='relu'))
model.add(Dropout(0.5))
# add the 2nd layer -- a hidden layer?
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
# add the 3rd layer -- an output layer
model.add(Dense(len(Y_train[0]), activation='softmax'))
# compile the model. SGD + Nesterov accel gradient yields good results 
# for the particular model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', 
              optimizer=sgd,
              metrics=['accuracy'])
print("Completed model compilation...")
# train model
hist = model.fit(np.array(x_train), 
                 np.array(Y_train),
                 epochs=200, 
                 batch_size=5,
                 verbose=1)
print("Completed model fitting...")
# cache model state into binary
model.save(CACHE/ "chatbot_model.h5", hist)
print("Completed model caching...")
